In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data0 = np.load('results/metrics_results_0.npy')
data1 = np.load('results/metrics_results_1.npy')
data2 = np.load('results/metrics_results_2.npy')
data3 = np.load('results/metrics_results_3.npy')
data4 = np.load('results/metrics_results_4.npy')
data5 = np.load('results/metrics_results_5.npy')
data6 = np.load('results/metrics_results_6.npy')
data7 = np.load('results/metrics_results_7.npy')
data8 = np.load('results/metrics_results_8.npy')
data9 = np.load('results/metrics_results_9.npy')

In [4]:
new_data = np.zeros_like(data9)

new_data[0] = data0[0]
new_data[1] = data1[1]
new_data[2] = data2[2]
new_data[3] = data3[3]
new_data[4] = data4[4]
new_data[5] = data5[5]
new_data[6] = data6[6]
new_data[7] = data8[8]
new_data[8] = data7[7]
new_data[9] = data9[9]

In [5]:
results = new_data  # shape: [n_models, n_runs, n_checkpoints, n_metrics]

orig_model_names = [
    "OT-CFM",
    "ICFM",
    "ICFM-Rej",
    "OTCFM-Iter20",
    "OTCFM-Iter200",
    "OTCFM-Iter10000",
    "OTCFM-Dir",
    "OTCFM-Dir-Tight",
    "OTCFM-Dir-PerfectPairing",
    "OTCFM-Iter6000",
]

orig_metrics_names = [
    "Avg Min Dist",
    "% Samples > 1.0",
    "Wasserstein Distance",
    "Coverage",
    "MMD",
    "MDD",
    "Normalized Wasserstein",
]

# Reorder models and rename for clarity
model_name_map = {
    "ICFM": "I-CFM",
    "OT-CFM": "OT-CFM",
    "ICFM-Rej": "I-CFM-Pruned",
    "OTCFM-Iter200": "Flow Model 200-iter",
    "OTCFM-Iter6000": "Flow Model 6k-iter",
    "OTCFM-Iter10000": "Flow Model 10k-iter",
    "OTCFM-Dir-PerfectPairing": "Directional Align (Normal) Perfect OT",
    "OTCFM-Dir": "Directional Align (Normal) OT-CFM",
    "OTCFM-Dir-Tight": "Directional Align (Tight) OT-CFM",
}

desired_model_order = [
    "ICFM",
    "OT-CFM",
    "ICFM-Rej",
    "OTCFM-Iter200",
    "OTCFM-Iter6000",
    "OTCFM-Iter10000",
    "OTCFM-Dir-PerfectPairing",
    "OTCFM-Dir",
    "OTCFM-Dir-Tight",
]

model_indices = [orig_model_names.index(m) for m in desired_model_order]
model_names = [model_name_map[m] for m in desired_model_order]
results = results[model_indices, :, :, :]  # Reorder results

In [6]:
# Select and rename metrics for clarity
metric_name_map = {
    "Avg Min Dist": "AMD ↓",
    "% Samples > 1.0": "Failure Rate ↓",
    "MDD": "MDD ↓",
    "Normalized Wasserstein": "Norm. Wassersterin ↓",
}

desired_metrics_order = [
    "Avg Min Dist",
    "% Samples > 1.0",
    "MDD",
    "Normalized Wasserstein",
]

metric_indices = [orig_metrics_names.index(m) for m in desired_metrics_order]
metrics_names = [metric_name_map[m] for m in desired_metrics_order]
results = results[:, :, :, metric_indices]  # Reorder results

In [7]:
n_models = len(model_names)
n_metrics = len(metrics_names)

min_table = []
for model_idx in range(n_models):
    row = []
    for metric_idx in range(n_metrics):
        # min over checkpoints per run: shape [n_runs, n_checkpoints] -> [n_runs]
        extrema_per_run = results[model_idx, :, :, metric_idx].min(axis=1)
        mean = np.mean(extrema_per_run)
        std = np.std(extrema_per_run)
        row.append(f"{mean:.4f} ± {std:.4f}")
    min_table.append(row)

df_min = pd.DataFrame(min_table, columns=metrics_names, index=model_names)
print("\n=== Min per Run → Avg ± Std (Selected Metrics) ===")
print(df_min.to_string())


=== Min per Run → Avg ± Std (Selected Metrics) ===
                                                 AMD ↓   Failure Rate ↓            MDD ↓ Norm. Wassersterin ↓
I-CFM                                  0.2271 ± 0.0371  2.4512 ± 1.0219  0.0908 ± 0.0297      1.4567 ± 0.1107
OT-CFM                                 0.3055 ± 0.0653  5.1367 ± 1.5010  0.0601 ± 0.0230      1.5237 ± 0.2779
I-CFM-Pruned                           0.1858 ± 0.0220  1.7188 ± 0.8414  0.1048 ± 0.0390      1.3587 ± 0.1183
Flow Model 200-iter                    0.3106 ± 0.0438  4.6094 ± 1.1201  0.0272 ± 0.0104      1.3393 ± 0.2184
Flow Model 6k-iter                     0.3250 ± 0.0567  6.0547 ± 1.2437  0.0166 ± 0.0051      1.3963 ± 0.1872
Flow Model 10k-iter                    0.3357 ± 0.0348  6.6602 ± 1.9210  0.0301 ± 0.0163      1.4654 ± 0.1675
Directional Align (Normal) Perfect OT  0.1540 ± 0.0071  0.7715 ± 0.2370  0.0012 ± 0.0006      0.5109 ± 0.0643
Directional Align (Normal) OT-CFM      0.2133 ± 0.0409  2.5000 ± 0.9